In [1]:
import pandas as pd
import numpy as np
import math
import xlsxwriter
import requests
import warnings

warnings.filterwarnings('ignore')

In [2]:
stocks = pd.read_csv('./sp_500_stocks.csv')

In [3]:
symbol = 'AAPL'
token = 'Tpk_059b97af715d417d9f49f50b51b1c448'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={token}'
data = requests.get(api_url).json()

In [4]:
price = data['latestPrice']
market_cap = data['marketCap']

In [5]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_df = pd.DataFrame(columns=my_columns)

In [6]:
final_df.append(pd.Series([symbol, price, market_cap, 'N/A/'], index=my_columns), ignore_index=True)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,150.57,2408251542663,N/A/


In [ ]:
# takes too long
final_df = pd.DataFrame(columns=my_columns)
for stock in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={token}'
    data = requests.get(api_url).json()
    final_df = final_df.append(pd.Series([stock, data['latestPrice'], data['marketCap'], 'N/A/'], index=my_columns), ignore_index=True)

In [8]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

In [9]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
final_df = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={token}'
    data = requests.get(batch_api_call_url).json()
    for symbol in  symbol_string.split(','):
        try:
            final_df = final_df.append(pd.Series([symbol, data[symbol]['quote']['latestPrice'], data[symbol]['quote']['marketCap'], 'N/A'], index=my_columns), ignore_index=True)
        except:
            pass
final_df

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,133.84,39628616795,N/A
1,AAL,14.35,9262389416,N/A
2,AAP,172.74,10610809992,N/A
3,AAPL,147.76,2371059987305,N/A
4,ABBV,148.35,259063297926,N/A
...,...,...,...,...
496,YUM,113.04,31555874278,N/A
497,ZBH,112.81,23663269115,N/A
498,ZBRA,273.42,13870191117,N/A
499,ZION,51.35,7418749394,N/A


In [10]:
portfolio_size = input('Enter the value of your portfolio: ')

try:
    val = float(portfolio_size)
except ValueError:
    print('Not a number \nPlease try again')
    portfolio_size = input('Enter the value of your portfolio: ')
    val = float(portfolio_size)

Enter the value of your portfolio: 10000000


In [11]:
position_size = val/len(final_df.index)
for i in range(len(final_df.index)):
    final_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_df.loc[i, 'Stock Price'])

In [12]:
writer = pd.ExcelWriter('recommand_trades.xlsx', engine='xlsxwriter')
final_df.to_excel(writer, 'Recommand Trades', index = False)

In [13]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)
dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)
integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [14]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['NUmber of Shares to Buy', integer_format],
}

In [15]:
for column in column_formats.keys():
    writer.sheets['Recommand Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['Recommand Trades'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

In [16]:
writer.save()

In [ ]:
writer.close()